In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

HEADERS ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}


import time

def scrape(url, interval, file_name):
    
    resp = requests.get(url, headers=HEADERS)
    content = BeautifulSoup(resp.content, 'lxml')
    soup = BeautifulSoup(resp.content, "html.parser")
    movie_containers = soup.find_all("div", {"class": "lister-item mode-advanced"})
    
    movie_list = []
    for movie in content.select('.lister-item-content'):
        time.sleep(interval)
        v = [d.text.strip() for d in movie.select('div:nth-of-type(3) a[href*="/name/"]')]
        try:
            data = {
                "title":movie.select('.lister-item-header')[0].get_text().strip(),
                "year":movie.select('.lister-item-year')[0].get_text().strip(),
                "certificate":movie.select('.certificate')[0].get_text().strip(),
                "time":movie.select('.runtime')[0].get_text().strip(),
                "genre":movie.select('.genre')[0].get_text().strip(),
                "rating":movie.select('.ratings-imdb-rating')[0].get_text().strip(),
                "metascore":movie.select('.ratings-metascore')[0].get_text().strip(),
                "simple_desc":movie.select('.text-muted')[2].get_text().strip(),
                "votes":movie.select('.sort-num_votes-visible')[0].get_text().strip(),
                "directors": v[0],
                "actors": v[1:3]
            }
        except IndexError:
            continue
    
        movie_list.append(data)

    
    dataframe = pd.DataFrame(movie_list)
    dataframe.to_csv(file_name)
    

url = "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&ref_=adv_prv"
url2 = "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=51&ref_=adv_nxt"
url3 = "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=101&ref_=adv_nxt"
url4 = "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=151&ref_=adv_nxt"
url5 = "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=201&ref_=adv_nxt"

scrape(url, 0, 'Top50.csv')
scrape(url2, 0, 'Top51-100.csv')
scrape(url3, 0, 'Top101-150.csv')
scrape(url4, 0, 'Top151-200.csv')
scrape(url5, 0, 'Top201-250.csv')


In [2]:

files = ["Top50.csv", "Top51-100.csv", "Top101-150.csv", "Top151-200.csv", "Top201-250.csv"]

temp = []

for file in files:
    df = pd.read_csv(file)
    temp.append(df)

com = pd.concat(temp)

com.to_csv("Top250.csv", index=False)